In [ ]:
%pip install pandas kaskada

In [ ]:
# Configure reloading of loaded modules (useful for the visualization function)
%reload_ext autoreload
%autoreload 2

In [ ]:
# Import pandas
import pandas as pd

In [ ]:
# Create the Kaskada Session
from kaskada.api.session import LocalBuilder
session = LocalBuilder().build()

In [ ]:
%reload_ext fenlmagic

In [ ]:
from kaskada import table

# Change this to the (relative) path to the generated input.
INPUT_PATH = '6x_1'

try:
    view.delete_view('ReviewsByItem')
except:
    pass

try:
  table.delete_table('Purchases')
except:
  pass

table.create_table(
  table_name = 'Purchases',
  time_column_name = 'time',
  entity_key_column_name = 'user',
  grouping_id = 'user',
)

table.load(
  table_name = 'Purchases',
  file = f'{INPUT_PATH}/purchases.parquet',
)

try:
  table.delete_table('PageViews')
except:
  pass

table.create_table(
  table_name = 'PageViews',
  time_column_name = 'time',
  entity_key_column_name = 'user',
  grouping_id = 'user',
)

table.load(
  table_name = 'PageViews',
  file = f'{INPUT_PATH}/page_views.parquet',
)

try:
  table.delete_table('Ratings')
except:
  pass

table.create_table(
  table_name = 'Reviews',
  time_column_name = 'time',
  entity_key_column_name = 'user',
  grouping_id = 'user',
)

table.load(
  table_name = 'Reviews',
  file = f'{INPUT_PATH}/reviews.parquet',
)

# Aggregation / History

In [ ]:
%%fenl --var=aggregate_history
{ 
    sum_amount: sum(Purchases.amount)
}

# Aggregation / Snapshot

In [ ]:
%%fenl --var=aggregate_snapshot --result-behavior final-results
{ 
    sum_amount: sum(Purchases.amount)
}

# Windowed Aggregation / History

In [ ]:
%%fenl --var=windowed_history
{ 
    sum_amount: sum(Purchases.amount, window=since(monthly()))
        # Hack to work-around https://github.com/kaskada-ai/kaskada/issues/297
        | if({ tick: monthly(), input: is_valid($input)} | not($input.tick | else(false)))
}

# Windowed Aggregation / Snapshot

In [ ]:
%%fenl --var=windowed_snapshot --result-behavior final-results
{ 
    sum_amount: sum(Purchases.amount, window=since(monthly()))
        # Hack to work-around https://github.com/kaskada-ai/kaskada/issues/297
        | if({ tick: monthly(), input: is_valid($input)} | not($input.tick | else(false)))
}

# Data-Defined Windowed Aggregation / History

In [ ]:
%%fenl --var=data_defined_history
let page_views_since_purchase_raw = count(PageViews, window=since(is_valid(Purchases)))
  | when(is_valid(Purchases))
in {
  result:  page_views_since_purchase_raw | mean()
}

# Temporal Join / Snapshot

In [ ]:
%%fenl --var=reviews_by_item
let reviews_by_item = Reviews | with_key($input.item)
in { average_score: reviews_by_item.rating | mean() }

In [ ]:
from kaskada import view

try:
    view.delete_view('ReviewsByItem')
except:
    pass

view.create_view(
    view_name = 'ReviewsByItem',
    expression = reviews_by_item.expression,
)

In [ ]:
%%fenl --var=temporal_join_snapshot --result-behavior final-results
{
  average_review: lookup(Purchases.item, ReviewsByItem.average_score)
}